# Модель предсказания срывов поставок на основе стакинга LightGBM

## Библиотеки и модули

Загрузка библиотек и модулей, используемых при решении. 
- Модель: Стак **LGBMClassifier** из библиотеки **LightGBM**
- Предобработчик данных для модели: **DataPreprocessor** (из модуля helper.data)
- Борьба с дисбалансом классов: **RandomOverSampler** (библиотека imbalanced-learn)
- Разделение на тренировочную и тестовую выборки, кросс-валидация, оценка решения: 
    - библиотека **scikit-learn**
    - **valid_predictions** (из модуля helper.validation)
- Работа с датасетом: библиотека **pandas** 
- Работа с файловой системой: модуль **os**

In [ ]:
# Модель
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score, train_test_split, StratifiedKFold
from sklearn.ensemble import StackingClassifier

# Пайплайн
from sklearn.pipeline import Pipeline
from helpers.data import DataPreprocessor
from helpers.validation import valid_predictions

# Данные
import os
import pandas as pd
from imblearn.over_sampling import RandomOverSampler

# Настройки вывода
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

## Загрузка датасетов

In [ ]:
# Пути
ROOT = os.getcwd()
TRAIN_DATASET = os.path.join(ROOT, '../data/train_AIC.csv')
BALANCED_DATASET = os.path.join(ROOT, '../data/balanced_train.csv')
TEST_DATASET = os.path.join(ROOT, '../data/test_AIC.csv')
SUBMISSION_PATH = os.path.join(ROOT, '../submissions/')

# Функции
def make_predictions(model: object | Pipeline, X_test: pd.DataFrame) -> None:
    """ Функция создания предсказаний для тестовой выборки.
    
    Обучает переданную модель на сбалансированном датасете, учитывая выбросы.
    Предсказания сохраняются с помощью функции save_submission.
    
    Параметры:
        model: Модель или пайплайн, используемый для обучения и предсказания меток
            тестовой выборки.
        X_test: Тестовая выборка, для которой будут сделаны предсказания.
            Экземпляр pandas.DataFrame."""
    
    # Загрузка сбалансированного датасета
    balanced_df = pd.read_csv(BALANCED_DATASET, index_col=0)
    
    # Обрезка негативных записей до числа позитивных
    first_negatives = balanced_df[balanced_df['y'] == 0][:balanced_df[balanced_df['y'] == 1]['y'].count()]
    balanced_df = pd.concat([balanced_df[balanced_df['y'] == 1], first_negatives])

    # Удаление выбросов
    balanced_df = balanced_df[balanced_df['Длительность'] < 400]
    balanced_df = balanced_df[(balanced_df['Сумма'] > 2) & (balanced_df['Сумма'] < 10)]
    balanced_df = balanced_df[balanced_df['До поставки'] < 300]
    balanced_df = balanced_df[balanced_df['Дней между 0_1'] < 400]
    balanced_df = balanced_df[balanced_df['Количество изменений после согласований'] < 2000]
    balanced_df = balanced_df[balanced_df['Количество'] < 300000]
    
    # Разделение независимых и независимых переменных
    X, y = balanced_df.iloc[:, :-1], balanced_df.iloc[:, -1]   

    # Обучение модели и создание предсказаний для тестовой выборки
    model.fit(X, y)
    preds = model.predict(X_test)

    # Сохранение предсказаний
    save_submission(preds, 'submission')
    
def save_submission(preds: list | pd.DataFrame | pd.arrays.PandasArray, subname: str) -> None:
    subname = os.path.join(SUBMISSION_PATH, f'{subname}.csv')
    submit_df = pd.DataFrame({'id': test_df.index, 'value': preds})
    submit_df.to_csv(subname, index=False)


# Загрузка датасетов
train_df = pd.read_csv(TRAIN_DATASET)
test_df = pd.read_csv(TEST_DATASET)

# Удаление дубликатов из тренировочной выборки
train_df = train_df.drop_duplicates()

# Разделение выборки на тренировочную и тестовую
X, y = train_df.iloc[:, :-1], train_df.iloc[:, -1]   
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Удаление выбросов из тренировочной выборки
X_train = pd.concat([X_train, y_train], axis=1) # Объединяем X_train и y_train
X_train = X_train[X_train['Длительность'] < 400]
X_train = X_train[(X_train['Сумма'] > 2) & (X_train['Сумма'] < 10)]
X_train = X_train[X_train['До поставки'] < 300]
X_train = X_train[X_train['Дней между 0_1'] < 400]
X_train = X_train[X_train['Количество изменений после согласований'] < 2000]
X_train = X_train[X_train['Количество'] < 300000]

# Отделяем метки классов от признаков
y_train = X_train['y']
X_train = X_train.drop('y', axis=1)

# Общий датасет для большего числа записей, используемых на кросс-валидации
X_general, y_general = train_df.iloc[:, :-1], train_df.iloc[:, -1]

In [ ]:
# Создание оверсемпл-датасета (отбор рандомных записей)
oversampler = RandomOverSampler(random_state=42)
X_res, y_res = oversampler.fit_resample(X_train, y_train)

# Параметры для моделей
lgbm_params = [
    {
    'learning_rate': 0.3, 
    'n_estimators': 1000, 
    'max_depth': 21, 
    'max_bin': 160, 
    'num_leaves': 128,
    'reg_lambda': 0.2,
    },
    {
    'learning_rate': 0.3, 
    'n_estimators': 1500, 
    'max_depth': 21, 
    'max_bin': 160, 
    'num_leaves': 128,
    'reg_lambda': 0.2,
    },
    {
    'learning_rate': 0.3, 
    'n_estimators': 1500, 
    'max_depth': 16, 
    'max_bin': 160, 
    'num_leaves': 96,
    'reg_lambda': 0.2,
    },
    {
    'learning_rate': 0.3, 
    'n_estimators': 1500, 
    'max_depth': 21, 
    'max_bin': 128, 
    'num_leaves': 32,
    'reg_lambda': 0.2,
    },
    {
    'learning_rate': 0.3, 
    'n_estimators': 1500, 
    'max_depth': 12, 
    'max_bin': 256, 
    'num_leaves': 128,
    'reg_lambda': 0.2,
    }
]

# Формирование списка моделей
models = []
for index, params in enumerate(lgbm_params):
    lgbm_model = LGBMClassifier(**params)
    models.append((f'lgbm_{index}', lgbm_model))

In [ ]:
# Предобработчик и модель
data_preprocessor = DataPreprocessor()
stack = StackingClassifier(estimators=models, verbose=1, cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=1708), n_jobs=-1)

# Пайплайн
pipeline = Pipeline([
    ('data', data_preprocessor),
    ('model', stack)
])

### Валидация на тестовой выборке

In [ ]:
# Обучение модели и получение предсказаний для тестовой выборки
pipeline.fit(X_train, y_train)
preds = pipeline.predict(X_test)

In [ ]:
# Вывод результатов валидации
valid_predictions(y_test, preds, pipeline.classes_)

## Получение предсказаний для AI Challenge

In [ ]:
# Предсказание для тестовой выборки AI Challenge
make_predictions(pipeline, test_df)